In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from collections import Counter
import math
import scipy.stats as st

# Init Variables

In [ ]:
NODE_NUMBER = 1000
MAX_EDGE_NUMBER = (NODE_NUMBER) * (NODE_NUMBER - 1) / 2
EDGE_PROBABILITY = 0.05
EDGE_NUMBER = int(MAX_EDGE_NUMBER * EDGE_PROBABILITY)  # Approximately 0.05 of max possible edge number
# GRAPH_NUMBER = 100
GRAPH_NUMBER = 20
# SIMULATION_NUMBER = 100
SIMULATION_NUMBER = 1
COLOR = ['red', 'green', 'blue', 'gray', 'yellow', 'brown', 'black']

# Functions

### Eigvals

###### Adjacency

In [ ]:
def get_adjacency_eigvals(graph):
    L = nx.adjacency_matrix(graph)
    e = np.linalg.eigvals(L.toarray())
    e = list(sorted(e))
    return e

###### Laplacian

In [ ]:
def get_laplacian_evigal(graph): # Laplacian matrix
    L = nx.laplacian_matrix(graph)
    e = np.linalg.eigvals(L.toarray())
    e = list(sorted(e))
    return e

### Spectral Gap

In [ ]:
def get_spectral_gap(graph):
    eigvals = get_adjacency_eigvals(graph)
    max_index = len(eigvals) - 1
    maximum = eigvals[max_index]
    second_max = eigvals[max_index - 1]
    diff = maximum - second_max
    return diff

### Algebraic Connectivity

In [ ]:
def get_algebraic_connectivity(graph):
    return nx.algebraic_connectivity(graph)

### check for connection with Algebraic Connectivity

In [ ]:
def is_graph_connected(graph):
    return get_algebraic_connectivity(graph) > 0

### Trace Power S

In [ ]:
def get_trace_power_s(graph, power=2): # number of walks with len s (power) in graph
    eigvals = get_adjacency_eigvals(graph)
    eigvals_powers = np.power(eigvals, power)
    summation = np.sum(eigvals_powers)
    return summation

### Phi s (Average Trace Power S)

In [ ]:
def get_phi_s(graph): # average number of walks with len s (power) in graph
    eigvals = get_adjacency_eigvals(graph)
    eigvals_powers = np.power(eigvals, power)
    summation = np.sum(eigvals_powers)
    avg = summation / len(eigvals)
    return summation

### Centrality of Global Subgraph

In [ ]:
def get_centrality_of_global_subgraph(graph):
    eigvals = get_adjacency_eigvals(graph)
    summation = sum([math.exp(value) for value in eigvals])
    return summation

### Average Eigvals

In [ ]:
def get_average_eigvals(graph):
    summation = get_centrality_of_global_subgraph(graph)
    ln = math.log(summation)
    return ln

## Automorphism

In [ ]:
def get_all_automorphism(graph): # automorphism is isomorphism for a graph with itself
    dictionary = nx.vf2pp_all_isomorphisms(graph, graph)
    return list(dictionary) # return a list of mapping (return a list of dictionary)

### Node Similarity (Vertex Transitivity)

In [ ]:
def is_node_similar(graph):
    automorphisms = get_all_automorphism(graph)
    for u,v in graph.edges:
        if not any(auto[u] == v for auto in automorphisms):
            return False
    return True

### Symmetry (Edge Transitivity)

In [ ]:
def is_symmetry(graph):
    automorphisms = get_all_automorphism(graph)
    for u, v in graph.edges:
        for x, y in graph.edges:
            if not any(auto[u] == x and auto[v] ==y for auto in automorphisms):
                return False
    return True

### Laplacian Energy

In [ ]:
def get_laplacian_energy(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_adjacency_eigvals(graph)
    laplacian_energy = sum([abs(value) for value in eigenvalues])
    return laplacian_energy

In [ ]:
def get_laplacian_energy2(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_adjacency_eigvals(graph)
    summation = sum([value if value > 0 else 0 for value in eigenvalues])
    laplacian_energy = summation * 2
    return laplacian_energy

In [ ]:
def get_laplacian_energy3(graph):
    if not nx.is_connected(graph):
        raise Exception("Graph Must Be Connected")
    if nx.is_directed(graph):
        raise Exception("Graph Must Be Undirected")
    eigenvalues = get_laplacian_evigal(graph)
    n = graph.number_of_nodes()
    m = graph.number_of_edges()
    constant = (2 * m) / n
    laplacian_energy = sum([abs(value - constant) for value in eigenvalues])
    return laplacian_energy

### Plotting Graphs

In [ ]:
def show_graph(graph, path=None, labels=False):
    pos = nx.circular_layout(graph)
    plt.figure(figsize = (12, 12))
    nx.draw_networkx(graph, pos, with_labels=labels)
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Degree Distribution

In [ ]:
def degree_distribution(graph, path=None, style='-o'):
    degrees = [graph.degree(n) for n in graph.nodes()]
    degrees = list(sorted(degrees))
    degree_freq_dic = Counter(degrees)
    x_axis = degree_freq_dic.keys()
    y_axis = degree_freq_dic.values()
    y_axis = np.array(list(y_axis)) / len(degrees)
    
    plt.title('Degree Distribution')
    plt.xlabel("Degree")
    plt.ylabel("Frequesncy")
    plt.plot(x_axis, y_axis, style, label='degree probability')
    
    upper_y = np.array([0, max(y_axis)])
    avg = np.average(degrees)
    upper_x = np.array([avg, avg])
    plt.plot(upper_x, upper_y, color='red', linestyle='-.', label='mean')
    plt.legend(loc='best') # setting best location for labels
    
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Double-Log

In [ ]:
def double_log(graph, path=None, style='-o'):
    degrees = [graph.degree(n) for n in graph.nodes()]
    degrees = list(sorted(degrees))
    degree_freq_dic = Counter(degrees)
    unique_degrees = list(degree_freq_dic.keys())
    frequency = list(degree_freq_dic.values())
    x_axis = np.log(unique_degrees)
    y_axis = np.log(frequency)
    y_axis = np.array(list(y_axis)) / len(degrees)
    plt.xlabel("Degree")
    plt.ylabel("Degree Distribution")
    plt.title('Double Log')
    plt.plot(x_axis, y_axis, style, label='degree distribution')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

## Comparing Plots

In [ ]:
def compare_datas(datas, labels, x_label='', y_label='', title='', style='-o', color=COLOR, path=None):
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    for i in range(len(datas)):
        x_axis = list(range(len(datas[i])))
        plt.plot(x_axis, datas[i], style, label=labels[i], color=color[i])
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

## Data Details

### c.i plots

In [ ]:
def coefficient_interval_plot(data, path=None, alpha=0.95):
    x = np.array([i for i in range(len(data))])
    y = np.array(data)
    # plotting
    plt.plot(y, x,'o', color='blue', label='data')
    
    # confidence intervals
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
#     print(f"\n{alpha*100} confidence interval {lower} and {upper}")
    
    # c.i upper & lower
    upper_y = np.array([0, len(data)])
    upper_x = np.array([upper, upper])
    plt.plot(upper_x, upper_y, color='red', linestyle='-.', label='upper c.i')
    
    lower_y = np.array([0, len(data)])
    lower_x = np.array([lower, lower])
    plt.plot(lower_x, lower_y, color='orange', linestyle='-.', label='lower c.i')
    
    ci_x = np.array([lower, upper])
    ci_y = np.array([0, 0])
    plt.plot(ci_x, ci_y, '-', color='green', label='c.i')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot2(data, path=None, alpha=0.95):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # confidence intervals
    ci = (1.0-alpha) * np.std(y) / np.mean(y)
    mean = np.mean(y)
    avg = [mean for i in range(len(data))]
    
    # Plot the confidence interval
    plt.fill_between(x, (avg-ci), (avg+ci), color='blue', alpha=0.1)
    plt.plot(x, (avg-ci), '--', color='blue', label='-*ci')
    plt.plot(x, (avg+ci), '--', color='blue', label='+*ci')
    plt.fill_between(x, (avg-2*ci), (avg+2*ci), color='green', alpha=.1)
    plt.plot(x, (avg-2*ci), '--', color='green', label='-2*ci')
    plt.plot(x, (avg+2*ci), '--', color='green', label='+2*ci')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot3(data, path=None, alpha=0.95):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # Define the confidence interval
    ci = (1.0-alpha) * np.std(y) / np.mean(y)
    
    # Plot the confidence interval
    plt.fill_between(x, (y-ci), (y+ci), color='blue', alpha=0.1)
    plt.plot(x, (y-2*ci), '--', color='blue', label='-*ci')
    plt.plot(x, (y+2*ci), '--', color='blue', label='+*ci')
    plt.fill_between(x, (y-2*ci), (y+2*ci), color='green', alpha=.1)
    plt.plot(x, (y-2*ci), '--', color='green', label='-2*ci')
    plt.plot(x, (y+2*ci), '--', color='green', label='+2*ci')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot4(data, path=None):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    y.astype('float64')
                 
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    low, high = st.norm.interval(alpha=0.9, loc=np.mean(data), scale=st.sem(data))
    low2, high2 = st.norm.interval(alpha=0.95, loc=np.mean(data), scale=st.sem(data))
    low3, high3 = st.norm.interval(alpha=0.99, loc=np.mean(data), scale=st.sem(data))
    
    # Plot the confidence interval
    plt.fill_between(x, [low3 for i in x], [high3 for i in x], color='green', alpha=0.1)
    plt.plot(x, [low3 for i in x], '--', color='green', label='alpha=0.99')
    plt.plot(x, [high3 for i in x], '--', color='green')
    
    plt.fill_between(x, [low2 for i in x], [high2 for i in x], color='orange', alpha=0.1)
    plt.plot(x, [low2 for i in x], '--', color='orange', label='0.95')
    plt.plot(x, [high2 for i in x], '--', color='orange')
    
    plt.fill_between(x, [low for i in x], [high for i in x], color='blue', alpha=0.1)
    plt.plot(x, [low for i in x], '--', color='blue', label='0.9')
    plt.plot(x, [high for i in x], '--', color='blue')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

In [ ]:
def coefficient_interval_plot5(data, path=None):
    x = np.array(list(range(len(data))))
    y = np.array(data)
    y.astype('float64')
                 
    # Plotting data
    plt.plot(x, y, '-o', color='red', label='data')
    
    # Plot the confidence interval
    alpha = 0.99
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
    
    plt.fill_between(x, [lower for i in x], [upper for i in x], color='green', alpha=0.1)
    plt.plot(x, [lower for i in x], '--', color='green', label='alpha=0.99')
    plt.plot(x, [upper for i in x], '--', color='green')
    
    alpha = 0.95
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
    
    plt.fill_between(x, [lower for i in x], [upper for i in x], color='orange', alpha=0.1)
    plt.plot(x, [lower for i in x], '--', color='orange', label='0.95')
    plt.plot(x, [upper for i in x], '--', color='orange')
    
    alpha = 0.9
    p = ((1.0-alpha)/2.0) * 100
    # percentile function returns the numbers which that percent of 
    # the array elements areless equal then that number
    lower =  np.percentile(y, p) 
    p = (alpha+((1.0-alpha)/2.0)) * 100
    upper =  np.percentile(y, p)
    
    plt.fill_between(x, [lower for i in x], [upper for i in x], color='blue', alpha=0.1)
    plt.plot(x, [lower for i in x], '--', color='blue', label='0.9')
    plt.plot(x, [upper for i in x], '--', color='blue')
    plt.legend(loc='best')
    if path != None:
        plt.savefig(path+'.png')
    plt.show()

### Mean & Variance

In [ ]:
def get_details(data):
    print('mean: ', np.mean(data))
    print('variance: ', np.var(data))

# RSRBG Generator

In [ ]:
def generate_rsrbg(d1, d2, n, seed=None):
    if (n * d2) % (d1 + d2) != 0:
        raise Exception(f"can't make graph with {n} nodes!")
        
    if seed != None:
        random.seed(seed)
    
    n1 = int((n * d2) / (d1 + d2))
    n2 = n - n1
    
    graph = nx.Graph()
    graph.add_nodes_from(list(range(n)))
    
    n1_stubs = list(range(n1)) * d1
    n2_stubs = list(range(n1, n)) * d2
    
    while len(n1_stubs) > 1:
        rnd1 = random.randint(0, len(n1_stubs) - 1)
        rnd2 = random.randint(0, len(n2_stubs) - 1)
        if graph.has_edge(n1_stubs[rnd1], n2_stubs[rnd2]):
            continue
        graph.add_edge(n1_stubs[rnd1], n2_stubs[rnd2])
        n1_stubs.pop(rnd1)
        n2_stubs.pop(rnd2)
    
    if graph.has_edge(n1_stubs[0], n2_stubs[0]):
        for u, v in graph.edges:
            n1_node = min([u, v])
            n2_node = max([u, v])
            if n1_node != n1_stubs[0] and n2_node != n2_stubs[0] and (not graph.has_edge(n1_node, n2_stubs[0])) and (not graph.has_edge(n1_stubs[0], n2_node)):
                graph.remove_edge(u, v)
                graph.add_edge(n1_node, n2_stubs[0])
                graph.add_edge(n1_stubs[0], n2_node)
                break
    else:
        graph.add_edge(n1_stubs[0], n2_stubs[0])
    return graph

# Making Some Random Examples

In [ ]:
seed_values = random.sample(range(1, 100000), GRAPH_NUMBER) # generating GRAPH_NUMBER unique random number to be used as seed
rsrbgs = []
rsrbgs.append(generate_rsrbg(2, 3, 5, seed_values[0]))
rsrbgs.append(generate_rsrbg(2, 3, 10, seed_values[1]))
rsrbgs.append(generate_rsrbg(4, 3, 7, seed_values[2]))
rsrbgs.append(generate_rsrbg(4, 3, 14, seed_values[3]))
rsrbgs.append(generate_rsrbg(2, 5, 7, seed_values[4]))
rsrbgs.append(generate_rsrbg(2, 5, 14, seed_values[5]))
rsrbgs.append(generate_rsrbg(4, 5, 9, seed_values[6]))
rsrbgs.append(generate_rsrbg(4, 5, 18, seed_values[7]))
rsrbgs.append(generate_rsrbg(6, 5, 22, seed_values[8]))
rsrbgs.append(generate_rsrbg(6, 5, 11, seed_values[9]))
rsrbgs.append(generate_rsrbg(6, 7, 13, seed_values[8]))
rsrbgs.append(generate_rsrbg(6, 8, 14, seed_values[9]))

In [ ]:
for graph in rsrbgs:
    show_graph(graph, labels=True)

### Degree Distribution

In [ ]:
for graph in rsrbgs:
    degree_distribution(graph)

# Algebraic Connectivity

In [ ]:
ac = []
for graph in rsrbgs:
    ac.append(get_algebraic_connectivity(graph))

In [ ]:
get_details(ac)

In [ ]:
coefficient_interval_plot(ac)
coefficient_interval_plot2(ac)
coefficient_interval_plot3(ac)
coefficient_interval_plot4(ac)
coefficient_interval_plot5(ac)

# Spectral Gap

In [ ]:
sg = []
for graph in rsrbgs:
    sg.append(get_spectral_gap(graph))

In [ ]:
get_details(sg)

In [ ]:
coefficient_interval_plot(sg)
coefficient_interval_plot2(sg)
coefficient_interval_plot3(sg)
try:
    coefficient_interval_plot4(sg)
except:
    plt.title('Erore Occured!') # Due To Complex Data Type as resualt of get_adjacency_eigvals function
    plt.show()
coefficient_interval_plot5(sg)

# Centrality Of Global Subgraph

In [ ]:
cgs = []
for graph in rsrbgs:
    cgs.append(get_centrality_of_global_subgraph(graph))

In [ ]:
get_details(cgs)

In [ ]:
coefficient_interval_plot(cgs)
coefficient_interval_plot2(cgs)
coefficient_interval_plot3(cgs)
coefficient_interval_plot4(cgs)
coefficient_interval_plot5(cgs)

# Average Eigvals

In [ ]:
ae = []
for graph in rsrbgs:
    ae.append(get_average_eigvals(graph))

In [ ]:
get_details(ae)

In [ ]:
coefficient_interval_plot(ae)
coefficient_interval_plot2(ae)
coefficient_interval_plot3(ae)
coefficient_interval_plot4(ae)
coefficient_interval_plot5(ae)

# Node Similarity

In [ ]:
ns = []
for graph in rsrbgs:
    ns.append(is_node_similar(graph))

In [ ]:
node_similar = (sum(ns) / len(ns)) * 100
print(f'node similar: {node_similar}%')

# Symmetry

In [ ]:
symmetry = []
for graph in rsrbgs:
    symmetry.append(is_symmetry(graph))

In [ ]:
sym = (sum(symmetry) / len(symmetry)) * 100
print(f'symmetry: {sym}%')

# Laplacian Energy

In [ ]:
lg = []
for graph in rsrbgs:
    lg.append(get_laplacian_energy(graph))

In [ ]:
get_details(lg)

In [ ]:
coefficient_interval_plot(lg)
coefficient_interval_plot2(lg)
coefficient_interval_plot3(lg)
coefficient_interval_plot4(lg)
coefficient_interval_plot5(lg)